# Basic jupyter notebook for shear bias computations

In [48]:
%run python/misc.py
%run python/simulations.py
%run python/shapes.py

In [49]:
#% run python/data.py
#download_HST_images()

In [67]:
# Job control:
job = misc.param()

# Re-run job if output file already exists?
job.re_run = False

# Set dry_run to True for test run
job.dry_run = True

In [63]:
# Small shear change for numerical derivative
dg = 0.02

In [64]:
# List of signs for shear change
# Four steps
g_steps = [(-1, 0), (0, -1), (1, 0), (0, 1)]

# Three steps, requires randomly changing signs
#g_step_list = [(-1, 0), (0, -1), (1, 1)]

In [65]:
# Dictionary and list of shears
g_dict = {}
for step in g_steps:
    g_dict[step] = (step[0] * dg, step[1] * dg)
g_values = g_dict.values()

### Galsim setup

In [66]:
# Files and directories
galsim_config_fname     = 'csc_multishear.yaml'
galsim_config_psf_fname = 'csc_psf.yaml'
galsim_config_dir       = 'config/galsim'
galsim_input_dir        = 'data/galsim/great3/control/space/constant'
galsim_output_base_dir  = 'output/galsim'

# Image sizes and numbers

# Number of postage stamps is nxy_tiles^2
nxy_tiles = 2
#nxy_tiles = 100

# Number of files with different constant shear and PSF
nfiles    =  4
#nfiles   = 200

In [68]:
# Set paths
galsim_config_path     = '{}/{}'.format(galsim_config_dir, galsim_config_fname)
galsim_config_psf_path = '{}/{}'.format(galsim_config_dir, galsim_config_psf_fname)

### Create image simulations by calling galsim

In [69]:
# TODO: Only run if output file does not exist
create_all_sims_great3(g_values, galsim_config_path, galsim_config_psf_path, galsim_input_dir, galsim_output_base_dir, nxy_tiles=nxy_tiles, nfiles=nfiles, job=job)

### Measure shapes

In [70]:
ksb_input_base_dir  = galsim_output_base_dir
ksb_output_base_dir = 'output/shapelens'
all_shapes_shapelens(g_values, ksb_input_base_dir, ksb_output_base_dir, nfiles, job=job)

### Get shear estimate results

In [86]:
import glob
psf_path = '{}/psf'.format(galsim_output_base_dir)
results = {}
for g in g_step_values:
    input_dir = '{}/{}'.format(ksb_output_base_dir, get_dir_name_shear(g))
    files     = glob.glob('{}/results*.txt'.format(input_dir))
    results[step] = get_ksb(files, psf_path)

In [98]:
R_output_dir = 'output/R'
#R = shear_response(results, dg, output_dir=R_output_dir)